# Design ideas/ goals for source code

## Generalized kalman filter for beleif state dynamics

### model components
- $x\in X$ 
- $y \in Y$
- $a \in A$
- $\nu \sim N(0,1)$
- $f: X \times V \times A \rightarrow  X \times Y$
- $R: X \times V \times A \rightarrow \bf R$
- $\delta = \frac{1}{1+d}$


### State transition function 

$f(\{x_{obs},x_{unobs},\nu_t \},a) + \epsilon_t = \{x_{obs}', x_{unobs}',y_t\}$ 

$\nu_t \sim N(\mathbf{0},\Sigma_{\nu})$

$\epsilon_t \sim N(\mathbf{0},\Sigma_{\{x,y \}})$

The model is defined by specifying a single function $f$ that takes the current state of the system $x_t$, the action chosen by the planer $a$ and a vector of i.i.d. random variables drawn from a standard normal distribution and returns the updated state vector $x_t$ and the observations made by the planner $y_t$. 

The action of the manager can effect both the state of the system $x$ and the observaitosn $y$.


### Solution

The beleif state transitions from the model are aproximated by evaluating the function $f$ at a set of Gaus-Hermite nodes scaled to reflect the current beleif state estiate and the error matrix as well as including the random variables $\nu$.

The values this proces returns are used to compute the mean $\{\hat{x},\hat{y}\}$ and the covariance $\Sigma_{\hat{x},\hat{y}}$ of the new state and observations. The updated covariance matrix for the state estimate can be computed by conditioning computing the conditioal distirubtion of the state variables given the observaitons with the matrix $\Sigma_{\hat{x},\hat{y}}$. The updated state estimate can be sampled by sampling an observaiton vector and condition with the covariance matrix $\Sigma_{\hat{x},\hat{y}}$. We can also integrate over the distirbution of possible esitmated states by using the samples of observaiton and weights fromthe Gaus-Hermite quadrature. 

A second posibility is to compute a list of updated estiamted states and covairance matrices by computing the mena and covairnce of the states $x'$ conditioning on the 

## Objects

### Base model
Only store data required to define the model
- dimensions of x and y
- dimensions of $\nu$
- covariance of purely additive noise
- set of action variables 
- joint state transition + observation function $f$
- rewards function 
- discount rate 

### Base solver
Stores data to define solution algorithm
- order of value funtion aproximation
- Value function
- Policy function
- order of quandrature aproximation
- x_nu_quad
- y_quad
- grid
- weights 

## Functions

### Simulate beleif state transition 
Simulates a joint transition of the systems state and the observers beleif state
- #### arguments
    - $\hat{x}$  estimated state 
    - $\Sigma_{\hat{x}}$ error matrix 
    - $x_0$ initial state
    - model::base_model
    - solver::base_solver
- #### values
    - $\hat{x}'$ updated state estimate 
    - $\Sigma_{\hat{x}'}$ updated error matrix 
    - $y$ observations
    - $r$ returns
    - $x'$ updated state 
    
### Integrate belief state transitions
Computes a set of updated beleif states $\{ \hat{x}, \Sigma_{\hat{x}} \}$ and a set of nodes to calcualte the expectation of the value function $V$ over the beleif states in the following period given te current beleif state and the chosen action. The function works by propogating uncertainty in the curent belief state, procees errors and observational errors to compute the prior predictive distribution of the observed variables $y$. A quadrature grid of "observations" is used to represent the sampling distirbution. New beleif states are computed for each of the quadrature nodes, but condition on that observation. 
- #### arguments 
    - $\hat{x}$  estimated state 
    - $\Sigma_{\hat{x}}$ error matrix 
    - model::base_model
    - solver::base_solver - includes quadrature to propogate uncertainty $x_0$ and $y$
        - x_nu_quad
        - y_quad
- #### values
    - nodes -  a vector of beleif states $\{ \hat{x}, \Sigma_{\hat{x}} \}$
    - weights - quadrature weights
- #### algorithm
    - update x_nu_quad to refelct the inital beleif state
    - compute $\{x,y\}$ for each quadrature node
    - compute mean $\{\hat{x},\hat{y}\}$ and covariance $\Sigma_{\{\hat{x},\hat{y}\}}$ using the samples and quadrature weights and additive noise $\Sigma_{add}$
    - update y_quad given $\hat{y}$ and $\Sigma_{\hat{y}}$
    - compute $\{\hat{x}, \Sigma_{\hat{x}} \}$ conditioning on the y_quad nodes
    - return vector of updated beleif states and y_quad weights


### Compute transition grid
Fills out a 3 dimensional array with beleif states. The first dimensions of the array indexes over the beleif state nodes used in the value function aproximation, the second over the possible actions and the third dimensions indexes over quadrature nodes.   
- #### arguments
    - model::base_model
    - solver::base_solver 
        - V - value funtion 
        - grid - pre allocated array to store state transition values 
        - x_nu_quad
        - y_quad   
- #### values 
    - updated solver object 


# Value functions 

In [324]:
include("MOMDPs.jl")
#include("../tests/MOMDPs.jl")

Main.MOMDPs

In [368]:
num_known_states = 1
num_unobserved_states = 1
number_of_observations = 1
state_transition = (x,fixed_policy,action) -> [0.9 0.1 1.0; 0.1 0.9 0.0; 0.9 0.1 0.0] * (x .+ action .+ fixed_policy)
nonadditive_noise_matrix = [0.1;;]
additive_noise_matrix = action -> [0.1 0.0 0.0; 0.0 0.01 0.0; 0.0 0.0 0.2]
total_states = num_known_states + num_unobserved_states
function fixed_policy_function(beleifState) 
    value = zeros(total_states+1)
    value[1] = 0.1*beleifState.estimated_states[1] 
    return value
end
actions = [[0.0,0.0,0.0],[0.1,0.0,0.0]]
reward_function = (x,fixed_policy,action) -> sum(x.^2)
discount_factor = 1.0/(1.0+0.05)
model=MOMDPs.init_model(num_known_states,num_unobserved_states,number_of_observations,
                  state_transition,nonadditive_noise_matrix,additive_noise_matrix,
                  fixed_policy_function,actions,reward_function,discount_factor)

Main.MOMDPs.Model(3, 2, 2, 1, 1, 1, 1, var"#161#162"(), [0.1;;], var"#163#164"(), true, fixed_policy_function, [[0.0, 0.0, 0.0], [0.1, 0.0, 0.0]], var"#165#166"(), 0.9523809523809523)

In [369]:
solver = MOMDPs.init_solver(model,5)
print(" ")

In [370]:
beleifState = MOMDPs.init_BeleifState([2.0],[0.1;;],[0.0]) 
MOMDPs.time_update!(solver,beleifState,actions[1],model)  
MOMDPs.sampling_distribution!(solver,actions[2],model)
MOMDPs.measurement_update!(beleifState,solver.observation_quadrature.nodes[100],solver,actions[1],model)

1×1 Matrix{Float64}:
 0.3926836736365851

# Compare with `KalmanFilters.jl`

In [371]:
using KalmanFilters




In [372]:
beleifState = MOMDPs.init_BeleifState([2.0],[0.1;;],[0.0]) 

Main.MOMDPs.BeleifState([2.0], [0.1;;], [0.0;;], [0.0])

In [373]:
x = zeros(3)
beleifState = MOMDPs.init_BeleifState([2.0],[0.1;;],[0.0]) 
x[2] = beleifState.estimated_states[1]
x[1] = beleifState.known_states[1]

Cov = zeros(3,3)
Cov[1,1] = 10^-5
Cov[2,2] = beleifState.covariance_matrix[1,1]
Cov[3,3] = model.nonadditive_noise_matrix[1,1]


F = x -> state_transition(x,model.fixed_policy_function(beleifState),actions[1])
Q = model.additive_noise_matrix(actions[1])
tu = time_update(x, Cov, F, Q)
get_state(tu)

3-element Vector{Float64}:
 0.3800000000628643
 1.8200000000651926
 0.3800000000628643

In [374]:
get_covariance(tu)

3×3 Matrix{Float64}:
 0.201008   0.0090009  0.0010081
 0.0090009  0.0910001  0.0090009
 0.0010081  0.0090009  0.201008

In [375]:
include("MOMDPs.jl")
solver = MOMDPs.init_solver(model,30)
beleifState = MOMDPs.init_BeleifState([2.0],[0.1;;],[0.0]) 
MOMDPs.time_update!(solver,beleifState,actions[1],model) 
solver.estimates_out

3-element Vector{Float64}:
 0.3799999999999969
 1.8199999999999865
 0.37999999999999673

In [376]:
solver.covariance_matrix_out

3×3 Matrix{Float64}:
 0.201  0.009  0.001
 0.009  0.091  0.009
 0.001  0.009  0.201

In [340]:
solver.covariance_matrix_in

2×2 Matrix{Float64}:
 0.1  0.0
 0.0  0.1

In [380]:
Cov = [1.0 0.0 0.5; 0.0 1.0 0.0; 0.5 0.0 1.0]
mu = [0.0,0.0,0.0]

Cov11 = Cov[1:2,1:2]
Cov21 = Cov[3:3,1:2]
Cov12 = Cov[1:2,3:3]
Cov22 = Cov[3:3,3:3]
mu1 = mu[1:2]
mu2 = mu[3:3]
observation = [1.0, 0.0]
mu2 + Cov21*inv(Cov11)*(observation .- mu1)

1-element Vector{Float64}:
 0.5

In [382]:
Cov22 .+ Cov21 * inv(Cov11) * Cov12

1×1 Matrix{Float64}:
 1.25